In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [3]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 24
# percentage of training set to use as validation
valid_size = 0.2

# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.Resize([299, 299]),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

0it [00:00, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [4]:
class EntryFlowLoop(nn.Module):
    def __init__(self,in_channels,out_channels,relu_start):
        super(EntryFlowLoop,self).__init__()
        self.relu_start=relu_start
        
        self.conv=nn.Conv2d(in_channels,out_channels,1,stride=2)
        self.bnc=nn.BatchNorm2d(out_channels)
        
        self.sc1a=nn.Conv2d(in_channels,in_channels,3,groups=in_channels,padding=1,bias=False)
        self.sc1b=nn.Conv2d(in_channels,out_channels,1,bias=False)
        self.bn1=nn.BatchNorm2d(out_channels)
        
        self.sc2a=nn.Conv2d(out_channels,out_channels,3,groups=out_channels,padding=1,bias=False)
        self.sc2b=nn.Conv2d(out_channels,out_channels,1,bias=False)
        self.bn2=nn.BatchNorm2d(out_channels)
        
        self.maxpool=nn.MaxPool2d(kernel_size=(3,3),stride=2,padding=1)
        self.relu=nn.ReLU()
    
    def forward(self,x):
                
        output=self.conv(x)
        output=self.bnc(output)
        
        if self.relu_start==True:
            x=self.relu(x)
        
        x=self.sc1b(self.sc1a(x))
        x=self.bn1(x)
        x=self.relu(x)
        
        x=self.sc2b(self.sc2a(x))
        x=self.bn2(x)
        x=self.maxpool(x)
        
        x+=output
        
        return x


class EntryFlowFinal(nn.Module):
    def __init__(self):
        super(EntryFlowFinal,self).__init__()
        
        self.c1=nn.Conv2d(3,32,3,stride=2)
        self.bn1=nn.BatchNorm2d(32)

        self.c2=nn.Conv2d(32,64,3)
        self.bn2=nn.BatchNorm2d(64)
        
        self.loop1=EntryFlowLoop(64,128,False)
        self.loop2=EntryFlowLoop(128,256,True)
        self.loop3=EntryFlowLoop(256,728,True)
        
        self.relu=nn.ReLU()
    
    def forward(self,x):
        x=self.c1(x)
        x=self.bn1(x)
        x=self.relu(x)
        
        x=self.c2(x)
        x=self.bn2(x)
        x=self.relu(x)
        
        x=self.loop1(x)
        x=self.loop2(x)
        x=self.loop3(x)
        
        return x

In [5]:
class MiddleFlow(nn.Module):
    def __init__(self,channels=728):
        super(MiddleFlow,self).__init__()
        
        self.sc1a=nn.Conv2d(728,728,3,groups=728,padding=1,bias=False)
        self.sc1b=nn.Conv2d(728,728,1,bias=False)
        
        self.sc2a=nn.Conv2d(728,728,3,groups=728,padding=1,bias=False)
        self.sc2b=nn.Conv2d(728,728,1,bias=False)
        
        self.sc3a=nn.Conv2d(728,728,3,groups=728,padding=1,bias=False)
        self.sc3b=nn.Conv2d(728,728,1,bias=False)
        
        
        self.bn1=nn.BatchNorm2d(728)
        self.bn2=nn.BatchNorm2d(728)
        self.bn3=nn.BatchNorm2d(728)
        self.relu=nn.ReLU()
        
    def forward(self,x):
        output=x
        
        x=self.sc1b(self.sc1a(self.relu(x)))
        x=self.bn1(x)
        x=self.sc2b(self.sc2a(self.relu(x)))
        x=self.bn2(x)
        x=self.sc3b(self.sc3a(self.relu(x)))
        x=self.bn3(x)
        
        x=x+output
        
        return x

    
class MiddleFlowFinal(nn.Module):
    def __init__(self):
        super(MiddleFlowFinal,self).__init__()
        self.cycle1=MiddleFlow()
        self.cycle2=MiddleFlow()
        self.cycle3=MiddleFlow()
        self.cycle4=MiddleFlow()
        self.cycle5=MiddleFlow()
        self.cycle6=MiddleFlow()
        self.cycle7=MiddleFlow()
        self.cycle8=MiddleFlow()
    
    
    
    def forward(self,x):
#         for i in range(8):
#             x=self.cycle(x)
        x=self.cycle1(x)
        x=self.cycle2(x)
        x=self.cycle3(x)
        x=self.cycle4(x)
        x=self.cycle5(x)
        x=self.cycle6(x)
        x=self.cycle7(x)
        x=self.cycle8(x)
        
        return x

In [6]:
class ExitFlowFinal(nn.Module):
    def __init__(self):
        super(ExitFlowFinal,self).__init__()
        
        self.c1=nn.Conv2d(728,1024,1,stride=2)
        self.bnc=nn.BatchNorm2d(1024)
        
        self.sc1a=nn.Conv2d(728,728,3,groups=728,padding=1,bias=False)
        self.sc1b=nn.Conv2d(728,728,1,bias=False)
        self.bn1=nn.BatchNorm2d(728)
        
        self.sc2a=nn.Conv2d(728,728,3,groups=728,padding=1,bias=False)
        self.sc2b=nn.Conv2d(728,1024,1,bias=False)
        self.bn2=nn.BatchNorm2d(1024)

        self.sc3a=nn.Conv2d(1024,1024,3,groups=1024,padding=1,bias=False)
        self.sc3b=nn.Conv2d(1024,1536,1,bias=False)
        self.bn3=nn.BatchNorm2d(1536)

        self.sc4a=nn.Conv2d(1536,1536,3,groups=1536,padding=1,bias=False)
        self.sc4b=nn.Conv2d(1536,2048,1,bias=False)
        self.bn4=nn.BatchNorm2d(2048)
        
        self.maxpool=nn.MaxPool2d(kernel_size=(3,3),stride=2,padding=1)
        self.relu=nn.ReLU()
        self.avgpool=nn.AvgPool2d(kernel_size=(10,10))
        
    def forward(self,x):
        output=self.c1(x)
        output=self.bnc(output)
        
        x=self.relu(x)
        x=self.sc1b(self.sc1a(x))
        x=self.bn1(x)
        
        x=self.relu(x)
        x=self.sc2b(self.sc2a(x))
        x=self.bn2(x)
        
        x=self.maxpool(x)
        x+=output
        
        x=self.sc3b(self.sc3a(x))
        x=self.bn3(x)        
        x=self.relu(x)
        
        x=self.sc4b(self.sc4a(x))
        x=self.bn4(x)        
        x=self.relu(x)
        
        x=self.avgpool(x)
        
        return x   

In [7]:
class Xception_implementation(nn.Module):
    def __init__(self):
        super(Xception_implementation,self).__init__()       
        
        self.entry=EntryFlowFinal()
        self.middle=MiddleFlowFinal()
        self.exit=ExitFlowFinal()
        
        self.fc1=nn.Linear(2048,512)
        self.ol=nn.Linear(512,275)
        
        self.relu=nn.ReLU()

    def forward(self,x):
        
        x=self.entry(x)
        x=self.middle(x)
        x=self.exit(x)
        
        x = x.view(x.size(0), -1) 
        x=self.relu(self.fc1(x))
        x=self.relu(self.ol(x))
        return x


model = Xception_implementation()
#print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()
! pip install torch-summary
from torchsummary import summary
summary(model,(3,299,299))

Layer (type:depth-idx)                   Output Shape              Param #
├─EntryFlowFinal: 1-1                    [-1, 728, 19, 19]         --
|    └─Conv2d: 2-1                       [-1, 32, 149, 149]        896
|    └─BatchNorm2d: 2-2                  [-1, 32, 149, 149]        64
|    └─ReLU: 2-3                         [-1, 32, 149, 149]        --
|    └─Conv2d: 2-4                       [-1, 64, 147, 147]        18,496
|    └─BatchNorm2d: 2-5                  [-1, 64, 147, 147]        128
|    └─ReLU: 2-6                         [-1, 64, 147, 147]        --
|    └─EntryFlowLoop: 2-7                [-1, 128, 74, 74]         --
|    |    └─Conv2d: 3-1                  [-1, 128, 74, 74]         8,320
|    |    └─BatchNorm2d: 3-2             [-1, 128, 74, 74]         256
|    |    └─Conv2d: 3-3                  [-1, 64, 147, 147]        576
|    |    └─Conv2d: 3-4                  [-1, 128, 147, 147]       8,192
|    |    └─BatchNorm2d: 3-5             [-1, 128, 147, 147]       256


Layer (type:depth-idx)                   Output Shape              Param #
├─EntryFlowFinal: 1-1                    [-1, 728, 19, 19]         --
|    └─Conv2d: 2-1                       [-1, 32, 149, 149]        896
|    └─BatchNorm2d: 2-2                  [-1, 32, 149, 149]        64
|    └─ReLU: 2-3                         [-1, 32, 149, 149]        --
|    └─Conv2d: 2-4                       [-1, 64, 147, 147]        18,496
|    └─BatchNorm2d: 2-5                  [-1, 64, 147, 147]        128
|    └─ReLU: 2-6                         [-1, 64, 147, 147]        --
|    └─EntryFlowLoop: 2-7                [-1, 128, 74, 74]         --
|    |    └─Conv2d: 3-1                  [-1, 128, 74, 74]         8,320
|    |    └─BatchNorm2d: 3-2             [-1, 128, 74, 74]         256
|    |    └─Conv2d: 3-3                  [-1, 64, 147, 147]        576
|    |    └─Conv2d: 3-4                  [-1, 128, 147, 147]       8,192
|    |    └─BatchNorm2d: 3-5             [-1, 128, 147, 147]       256


In [8]:
import torch.optim as optim

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(),lr=0.02, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr = 0.00005)

# Trying different lr and Optimizers to suits the data

### SGD, lr=0.0005  
100 train loss =  4.293675837516784  
200 train loss =  3.5160562169551848  
300 train loss =  3.1869335826237997  
400 train loss =  2.995716138780117  
500 train loss =  2.8791215178966523  
600 train loss =  2.778849496046702  

### SGD, lr=0.001  
100 train loss =  3.5865376210212707  
200 train loss =  3.1011905866861342  
300 train loss =  2.853240623474121  
400 train loss =  2.722959086894989  
500 train loss =  2.635439955234528  
600 train loss =  2.568245601654053  

### SGD, lr=0.005  
100 train loss =  3.3242822325229646  
200 train loss =  2.8887323570251464  
300 train loss =  2.7026949417591095  
400 train loss =  2.5906972613930703  
500 train loss =  2.4937503309249878  
600 train loss =  2.4266555760304134  

### SGD, lr=0.007  
100 train loss =  3.410749969482422  
200 train loss =  2.9440751534700396  
300 train loss =  2.6484226047992707  
400 train loss =  2.4255268818140028  
500 train loss =  2.2726909840106964  

### SGD, lr=0.01  

100 train loss =  3.286306972503662  
200 train loss =  2.927250148653984  
300 train loss =  2.7289543612798055  
400 train loss =  2.6260555094480513  
500 train loss =  2.5271109757423402  
600 train loss =  2.3860551651318866  

### SGD, lr=0.02  

100 train loss =  2.632558946609497  
200 train loss =  2.322135624289513  
300 train loss =  2.1878742357095082  
400 train loss =  2.100518697798252  
500 train loss =  2.0330836935043335  
600 train loss =  1.9819465831915537  



### SGD, lr=0.03  
100 train loss =  3.141713446378708  
200 train loss =  2.8451237058639527  
300 train loss =  2.7730438776810966  
400 train loss =  2.695722301900387  
500 train loss =  2.6434776804447173  
600 train loss =  2.5930514949560166  

## Optimizer change to Adam  

### Adam, lr=0.1

100 train loss =  21.78967511177063  
200 train loss =  13.34359126329422  

### Adam, lr=0.05  
100 train loss =  10.229436330795288  
200 train loss =  6.703847949504852  

### Adam, lr=0.01  
100 train loss =  4.773248999118805  
200 train loss =  4.5610452401638035  
300 train loss =  4.43212244908015  
400 train loss =  4.393921576142311  
500 train loss =  4.357113010406494  
600 train loss =  4.320134338537852  

### Adam, lr=0.005  
100 train loss =  4.843475396633148  
200 train loss =  4.747493587732315  
300 train loss =  4.689231255849203  
400 train loss =  4.645357070565224  

### Adam, lr=0.001  
100 train loss =  3.5790255165100096  
200 train loss =  3.4403728437423706  
300 train loss =  3.3438539365927378  
400 train loss =  3.2561190494894983  
500 train loss =  3.219608674287796  

### Adam, lr=0.0005  
100 train loss =  3.8323942432403566  
200 train loss =  3.714814779758453  
300 train loss =  3.605844489336014  
400 train loss =  3.529643462896347  
500 train loss =  3.4812713096141816

### Adam, lr=0.0001  
100 train loss =  3.6713942432403566  
200 train loss =  3.414814779758453  
300 train loss =  3.305844489336014  
400 train loss =  3.229643462896347  
500 train loss =  3.1812713096141816  

### Adam, lr=0.00005  
100 train loss =  3.7643074476718903  
200 train loss =  3.2872815561294555  
300 train loss =  3.0677139552434287  
400 train loss =  2.9719791358709333  
500 train loss =  2.8868279342651366  





In [9]:
n_epochs = 25
valid_loss_min = np.Inf # track change in validation loss
batch_number=0
for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    
    for data, target in train_loader:
        batch_number+=1
        
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss

        #output =output.reshape(20,-1)
        #print(output.shape)
        loss = criterion(output, target)

        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        if batch_number%400==0:
            print(batch_number)
            torch.save(model.state_dict(), 'model_cifar_1.pt')

    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

400
800
1200
1600
Epoch: 1 	Training Loss: 1.370068 	Validation Loss: 0.251383
Validation loss decreased (inf --> 0.251383).  Saving model ...
2000
2400
2800
3200
Epoch: 2 	Training Loss: 0.842680 	Validation Loss: 0.179407
Validation loss decreased (0.251383 --> 0.179407).  Saving model ...
3600
4000
4400
4800
Epoch: 3 	Training Loss: 0.614722 	Validation Loss: 0.154220
Validation loss decreased (0.179407 --> 0.154220).  Saving model ...
5200
5600
6000
6400
Epoch: 4 	Training Loss: 0.468443 	Validation Loss: 0.117426
Validation loss decreased (0.154220 --> 0.117426).  Saving model ...
6800
7200
7600
8000
Epoch: 5 	Training Loss: 0.362242 	Validation Loss: 0.099673
Validation loss decreased (0.117426 --> 0.099673).  Saving model ...
8400
8800
9200
9600
10000
Epoch: 6 	Training Loss: 0.293766 	Validation Loss: 0.099219
Validation loss decreased (0.099673 --> 0.099219).  Saving model ...
10400
10800
11200
11600
Epoch: 7 	Training Loss: 0.228736 	Validation Loss: 0.093741
Validation loss 

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'model_cifar_1.pt')

In [ ]:
model.load_state_dict(torch.load('model_cifar.pt'))

In [9]:
model1 = Xception_implementation()
model2 = Xception_implementation()


# move tensors to GPU if CUDA is available
if train_on_gpu:
    model1.cuda()
    model2.cuda()

model1.load_state_dict(torch.load('../input/saved-models-cifar10/model_cifar (7).pt'))
model2.load_state_dict(torch.load('../input/saved-models-cifar10/model_cifar_1.pt'))

<All keys matched successfully>

In [12]:
model=model2
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    batch_size=data.size(0)
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),

    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.537241

Test Accuracy of airplane: 83% (836/1000)
Test Accuracy of automobile: 92% (924/1000)
Test Accuracy of  bird: 78% (788/1000)
Test Accuracy of   cat: 53% (538/1000)
Test Accuracy of  deer: 82% (821/1000)
Test Accuracy of   dog: 88% (881/1000)
Test Accuracy of  frog: 89% (894/1000)
Test Accuracy of horse: 92% (922/1000)
Test Accuracy of  ship: 93% (939/1000)
Test Accuracy of truck: 95% (953/1000)

Test Accuracy (Overall): 84% (8496/10000)


In [13]:
model=model1
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    batch_size=data.size(0)
    for i in range(batch_size):
        
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.471307

Test Accuracy of airplane: 84% (848/1000)
Test Accuracy of automobile: 94% (941/1000)
Test Accuracy of  bird: 68% (687/1000)
Test Accuracy of   cat: 80% (800/1000)
Test Accuracy of  deer: 82% (822/1000)
Test Accuracy of   dog: 79% (790/1000)
Test Accuracy of  frog: 93% (939/1000)
Test Accuracy of horse: 84% (848/1000)
Test Accuracy of  ship: 95% (955/1000)
Test Accuracy of truck: 90% (905/1000)

Test Accuracy (Overall): 85% (8535/10000)


### We gat a accuracy of 85 % !!